In [1]:
%pip install langchain langchain_openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


Oftentimes you may want to experiment with, or even expose to the end user, multiple different ways of doing things. In order to make this experience as easy as possible, we have defined two methods.

First, a `configurable_fields` method. This lets you configure particular fields of a runnable.

Second, a `configurable_alternatives` method. With this method, you can list out alternatives for any particular runnable that can be set during runtime.

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

In [4]:
model.invoke("pick a random number")

AIMessage(content='27', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8d6f2000-daad-425f-9fec-592beaf5b89f-0', usage_metadata={'input_tokens': 11, 'output_tokens': 1, 'total_tokens': 12})

In [5]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("pick a random number")

AIMessage(content='27', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-785c596a-7def-40ca-8789-1164bbbce721-0', usage_metadata={'input_tokens': 11, 'output_tokens': 1, 'total_tokens': 12})

## Configuring Prompts

In [6]:
llm = ChatOpenAI(temperature=0)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

In [7]:
# By default it will write a joke
chain.invoke({"topic": "bears"})

AIMessage(content="Why did the bear break up with his girlfriend?\nBecause he couldn't bear the relationship anymore!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 13, 'total_tokens': 32, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f45c6bff-2dcf-48fd-88b9-1851b11b7777-0', usage_metadata={'input_tokens': 13, 'output_tokens': 19, 'total_tokens': 32})

In [8]:
# We can configure it write a poem
chain.with_config(configurable={"prompt": "poem"}).invoke({"topic": "bears"})

AIMessage(content="In the forest deep and wild,\nWhere the trees stand tall and proud,\nThere roams a creature fierce and mild,\nThe mighty bear, so strong and loud.\n\nWith fur as dark as midnight's cloak,\nAnd eyes that gleam with ancient wisdom,\nHe moves with grace, his presence spoke,\nIn the silence of the woodland kingdom.\n\nHe hunts for fish in rushing streams,\nAnd berries ripe on bushes low,\nHis roar echoes through the moonlit beams,\nA reminder of the wild's untamed flow.\n\nBut do not fear this noble beast,\nFor he is part of nature's grand design,\nA symbol of strength and peace,\nIn the wilderness, where he reigns divine.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 137, 'prompt_tokens': 13, 'total_tokens': 150, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cedc2873-1a3e-46cf-908f-256bb84d40a2-0', 

## Saving Configurations

In [10]:
openai_joke = chain.with_config(configurable={"llm": "openai"})

In [11]:
openai_joke

RunnableBinding(bound=RunnableConfigurableAlternatives(default=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}'), which=ConfigurableField(id='prompt', name=None, description=None, annotation=None, is_shared=False), alternatives={'poem': PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Write a short poem about {topic}')}, default_key='joke', prefix_keys=False)
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7a63b425e560>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a63b4290a30>, root_client=<openai.OpenAI object at 0x7a63b425eb60>, root_async_client=<openai.AsyncOpenAI object at 0x7a63b425e200>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={}, config={'configurable': {'llm': 'openai'}}, config_factories=[])